In [24]:
!pip install ipywidgets --upgrade

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.4 kB ? eta -:--:--
   -------- ------------------------------ 30.7/139.4 kB 435.7 kB/s eta 0:00:01
   ----------------- --------------------- 61.4/139.4 kB 465.5 kB/s eta 0:00:01
   ---------------------- ---------------- 81.9/139.4 kB 416.7 kB/s eta 0:00:01
   ------------------------- ------------- 92.2/139.4 kB 403.5 kB/s eta 0:00:01
   ------------------------------------ - 133.1/139.4 kB 462.0 kB/s eta 0:00:01
   -------------------------------------- 139.4/139.4 kB 434.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ----- --------------------------------- 30.7/215.0 kB 660.6 kB/s eta 0:00:01
   ----------- --------------------------- 61.4/215.0 kB 656.4 kB/s eta 0:00:

In [35]:
pip install -U ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Подключение библиотек
import pandas as pd
import networkx as nx
from pyvis.network import Network
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, widgets

In [3]:
data = pd.read_csv("imdb_movies.csv")

In [4]:
#Обработка датасета от пустых значений 
data = data.dropna()

data.reset_index(drop=True, inplace=True)

In [5]:
data.head(5)

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [6]:
# Пользователь вводит имя актера
actor_name = input("Введите имя актера: ")
actor_movies = data[data['crew'].str.contains(actor_name, na=False)]

Введите имя актера:  Jake Gyllenhaal


In [7]:
# Вывод найденных фильмов
if not actor_movies.empty:
    print(f"Фильмы, в которых участвует актер {actor_name}:")
    for idx, row in actor_movies.iterrows():
        print(f"Название: {row['names']}")
        print(f"Дата выпуска: {row['date_x']}")
        print(f"Жанр: {row['genre']}")
        print(f"Рейтинг: {row['score']}")
        print("-----")
else:
    print(f"Фильмы с участием актера {actor_name} не найдены.")

Фильмы, в которых участвует актер Jake Gyllenhaal:
Название: Strange World
Дата выпуска: 11/24/2022 
Жанр: Animation, Family, Adventure, Science Fiction
Рейтинг: 65.0
-----
Название: Brothers
Дата выпуска: 03/18/2010 
Жанр: Drama, Thriller, War
Рейтинг: 73.0
-----
Название: Brothers
Дата выпуска: 03/18/2010 
Жанр: Drama, Thriller, War
Рейтинг: 73.0
-----
Название: Spider-Man: Far From Home
Дата выпуска: 07/01/2019 
Жанр: Action, Adventure, Science Fiction
Рейтинг: 75.0
-----
Название: Spirit Untamed
Дата выпуска: 06/12/2021 
Жанр: Animation, Adventure, Family, Western
Рейтинг: 73.0
-----
Название: Ambulance
Дата выпуска: 04/07/2022 
Жанр: Thriller, Action
Рейтинг: 68.0
-----
Название: Southpaw
Дата выпуска: 08/20/2015 
Жанр: Action, Drama
Рейтинг: 74.0
-----
Название: Prisoners
Дата выпуска: 10/17/2013 
Жанр: Drama, Thriller, Crime
Рейтинг: 81.0
-----
Название: Love & Other Drugs
Дата выпуска: 12/16/2010 
Жанр: Drama, Comedy, Romance
Рейтинг: 70.0
-----
Название: Prince of Persia: The 

In [10]:
def create_and_show_graph(actor_movies, min_rating=0, max_rating=100):
    G = nx.Graph()
    
    for idx, row in actor_movies.iterrows():
        # Добавляем только если рейтинг находится в указанном диапазоне
        if min_rating <= row['score'] <= max_rating:
            G.add_node(row['names'], date=row['date_x'], genre=row['genre'], score=row['score'])
    
    # Создание рёбер на основе общих актеров, исключая искомого актера
    for movie1 in G.nodes():
        for movie2 in G.nodes():
            if movie1 != movie2:
                # Находим актеров для записи и разбиваем на список актеров
                actors_movie1 = set(data[data['names'] == movie1]['crew'].iloc[0].split(", "))
                actors_movie2 = set(data[data['names'] == movie2]['crew'].iloc[0].split(", "))
                # Пересечение общих актеров
                common_actors = actors_movie1.intersection(actors_movie2)
                
                common_actors = [actor for actor in common_actors if actor != actor_name]  # Исключаем искомого актера
                # Если общие актеры найдены, то добавляем ребро в граф 
                if common_actors:
                    G.add_edge(movie1, movie2, common_actors=common_actors)

    # Создаем объект графа
    nt = Network(height='800px', width='100%', notebook=True)
    nt.from_nx(G)
    
    # Добавление информации при наведении на ребро
    for edge in nt.edges:
        common_actors = ', '.join(edge['common_actors'])
        edge["title"] = f"Общие актеры: {common_actors}"
    
    # Добавление информации при наведении на узел
    for node in nt.nodes:
        node_info = f"Дата выпуска: {node['date']}. Жанр:{node['genre']}. Рейтинг: {node['score']}"
        node["title"] = node["label"] + "<br>" + node_info
    
    # Отображение графа
    nt.show('graph.html')
    print(f"Граф фильмов с участием актера {actor_name} и рейтингом от {min_rating} до {max_rating} создан. Откройте файл graph.html для просмотра.")
    # Сохраняем в файл
    nt.save_graph("graph.html")


# Вызов функции создания и отображения графа для начального актера и заданных диапазонов рейтинга
create_and_show_graph(actor_movies, min_rating=0, max_rating=100) 

graph.html
Граф фильмов с участием актера Jake Gyllenhaal и рейтингом от 0 до 100 создан. Откройте файл graph.html для просмотра.


In [11]:
# Создание виджетов для фильтрации по минимальному и максимальному рейтингу и кнопка обновления
min_rating_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description='Мин. рейтинг:')
max_rating_slider = widgets.FloatSlider(value=100, min=0, max=100, step=0.1, description='Макс. рейтинг:')
update_button = widgets.Button(description="Обновить граф")

In [12]:
def update_graph_with_max_rating(b):
    create_and_show_graph(actor_movies, min_rating=min_rating_slider.value, max_rating=max_rating_slider.value)

# Привязка функции-обработчика к кнопке
update_button.on_click(update_graph_with_max_rating)

# Отображение виджетов
display(min_rating_slider)
display(max_rating_slider)
display(update_button)

FloatSlider(value=0.0, description='Мин. рейтинг:')

FloatSlider(value=100.0, description='Макс. рейтинг:')

Button(description='Обновить граф', style=ButtonStyle())